[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danpele/Time-Series-Analysis/blob/main/chapter10_lecture_notebook.ipynb)

---

# Chapter 10: Comprehensive Review

**Complete Time Series Analysis with Real Data**

**Course:** Time Series Analysis and Forecasting  
**Program:** Bachelor program, Faculty of Cybernetics, Statistics and Economic Informatics, Bucharest University of Economic Studies, Romania  
**Academic Year:** 2025-2026

---

## Learning Objectives

This comprehensive review demonstrates the complete time series analysis workflow using **real data**:

1. **Case Study 1: Bitcoin** - Cryptocurrency volatility analysis with ARIMA-GARCH
2. **Case Study 2: Sunspots** - Long-cycle seasonal data (11-year Schwabe cycle)
3. **Case Study 3: US Unemployment** - Economic data with COVID-19 structural break

We will apply ALL methods covered in the course:
- Stationarity testing (ADF, KPSS)
- ARIMA/SARIMA modeling
- GARCH for volatility clustering
- Prophet with changepoint detection
- Fourier terms for long seasonality
- Model comparison and evaluation

## Setup and Imports

In [ ]:
# Install required packages (for Colab)
import sys
if 'google.colab' in sys.modules:
    !pip install prophet arch statsmodels yfinance pandas-datareader -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Statistical tests and models
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose

# GARCH
try:
    from arch import arch_model
    HAS_ARCH = True
except ImportError:
    HAS_ARCH = False
    print("arch not installed. Install with: pip install arch")

# Prophet
try:
    from prophet import Prophet
    HAS_PROPHET = True
except ImportError:
    HAS_PROPHET = False
    print("Prophet not installed. Install with: pip install prophet")

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Plotting style - CONSISTENT WITH OTHER CHAPTERS
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False

COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666'}

print("Setup complete!")
print(f"ARCH/GARCH available: {HAS_ARCH}")
print(f"Prophet available: {HAS_PROPHET}")

## Real Data Loading Functions

We use actual data from reliable sources:
- **Bitcoin**: Yahoo Finance via `yfinance`
- **Sunspots**: Classic dataset from `statsmodels`
- **US Unemployment**: Federal Reserve Economic Data (FRED)

In [ ]:
def get_bitcoin_data():
    """Fetch REAL Bitcoin daily prices from Yahoo Finance (2019-2024)"""
    try:
        import yfinance as yf
        btc = yf.download('BTC-USD', start='2019-01-01', end='2024-01-01', progress=False)
        df = pd.DataFrame({
            'ds': btc.index,
            'price': btc['Close'].values
        }).reset_index(drop=True)
        df['returns'] = df['price'].pct_change() * 100
        print(f"Bitcoin: Loaded {len(df)} days of REAL data from Yahoo Finance")
        return df
    except Exception as e:
        print(f"Could not load Bitcoin data: {e}")
        print("Using backup simulated data...")
        # Fallback to simulated data
        np.random.seed(42)
        dates = pd.date_range('2019-01-01', '2024-01-01', freq='D')
        price = 3700
        prices = [price]
        for i in range(1, len(dates)):
            date = dates[i]
            if date < pd.Timestamp('2020-03-01'):
                drift, vol = 0.001, 0.03
            elif date < pd.Timestamp('2020-04-01'):
                drift, vol = -0.02, 0.08
            elif date < pd.Timestamp('2021-11-01'):
                drift, vol = 0.003, 0.04
            elif date < pd.Timestamp('2023-01-01'):
                drift, vol = -0.002, 0.045
            else:
                drift, vol = 0.0015, 0.025
            ret = drift + vol * np.random.randn()
            price = max(prices[-1] * (1 + ret), 1000)
            prices.append(price)
        df = pd.DataFrame({'ds': dates, 'price': prices})
        df['returns'] = df['price'].pct_change() * 100
        return df


def get_sunspot_data():
    """Fetch REAL monthly sunspot numbers from statsmodels (1749-present)"""
    try:
        import statsmodels.api as sm
        sunspots = sm.datasets.sunspots.load_pandas().data
        # Use data from 1900 onwards for cleaner analysis
        sunspots = sunspots[sunspots['YEAR'] >= 1900].copy()
        # Create proper date index (yearly data)
        sunspots['ds'] = pd.to_datetime(sunspots['YEAR'].astype(int), format='%Y')
        sunspots = sunspots.rename(columns={'SUNACTIVITY': 'y'})
        sunspots = sunspots[['ds', 'y']].reset_index(drop=True)
        print(f"Sunspots: Loaded {len(sunspots)} years of REAL data from statsmodels")
        return sunspots
    except Exception as e:
        print(f"Could not load sunspot data: {e}")
        print("Using backup simulated data...")
        np.random.seed(123)
        dates = pd.date_range('1900-01-01', '2023-12-01', freq='YS')
        cycle_period = 11  # 11-year cycle
        sunspots = []
        for i in range(len(dates)):
            phase = (i % cycle_period) / cycle_period
            cycle_value = 150 * np.sin(np.pi * phase) ** 1.5
            cycle_num = i // cycle_period
            cycle_amplitude = [1.0, 0.8, 1.1, 0.9, 1.2, 0.7, 1.0, 0.85, 1.15, 0.95, 1.1][cycle_num % 11]
            cycle_value *= cycle_amplitude
            noise = np.random.normal(0, 15)
            sunspots.append(max(0, cycle_value + noise))
        return pd.DataFrame({'ds': dates, 'y': sunspots})


def get_unemployment_data():
    """Fetch REAL US Unemployment Rate from FRED (2015-2023)"""
    try:
        import pandas_datareader as pdr
        unemp = pdr.get_data_fred('UNRATE', start='2015-01-01', end='2023-12-31')
        df = pd.DataFrame({
            'ds': unemp.index,
            'y': unemp['UNRATE'].values
        }).reset_index(drop=True)
        print(f"US Unemployment: Loaded {len(df)} months of REAL data from FRED")
        return df
    except Exception as e:
        print(f"Could not load unemployment data: {e}")
        print("Using backup simulated data...")
        np.random.seed(456)
        dates = pd.date_range('2015-01-01', '2023-12-01', freq='MS')
        unemployment = []
        for i, date in enumerate(dates):
            if date < pd.Timestamp('2020-03-01'):
                base = 5.5 - (i / 60) * 2
                unemployment.append(max(3.5, base + np.random.normal(0, 0.1)))
            elif date < pd.Timestamp('2020-05-01'):
                unemployment.append(4.4 if date.month == 3 else 14.7)
            elif date < pd.Timestamp('2021-01-01'):
                months_since = (date.year - 2020) * 12 + date.month - 4
                rate = 14.7 - months_since * 1.2
                unemployment.append(max(6.7, rate + np.random.normal(0, 0.2)))
            elif date < pd.Timestamp('2023-01-01'):
                months_since = (date.year - 2021) * 12 + date.month
                rate = 6.7 - months_since * 0.15
                unemployment.append(max(3.5, rate + np.random.normal(0, 0.1)))
            else:
                unemployment.append(3.5 + np.random.normal(0, 0.1))
        return pd.DataFrame({'ds': dates, 'y': unemployment})


print("Real data loading functions defined!")
print("- Bitcoin: Yahoo Finance API")
print("- Sunspots: statsmodels classic dataset")
print("- US Unemployment: FRED API")

---
# Case Study 1: Bitcoin Volatility Analysis

Cryptocurrency data is characterized by:
- **Extreme volatility** (much higher than traditional assets)
- **Regime changes** (bull markets, crypto winters)
- **Fat tails** (more extreme events than normal distribution)
- **Volatility clustering** (GARCH is essential)

In [ ]:
# Load REAL Bitcoin data
btc = get_bitcoin_data()

print("\nBitcoin Data Overview")
print("="*50)
print(f"Period: {btc['ds'].min().date()} to {btc['ds'].max().date()}")
print(f"Observations: {len(btc)} days")
print(f"\nPrice Statistics:")
print(f"  Min: ${btc['price'].min():,.2f}")
print(f"  Max: ${btc['price'].max():,.2f}")
print(f"\nReturn Statistics (Daily %):")
print(f"  Mean: {btc['returns'].mean():.4f}%")
print(f"  Std: {btc['returns'].std():.4f}%")
print(f"  Skewness: {btc['returns'].skew():.4f}")
print(f"  Kurtosis: {btc['returns'].kurtosis():.4f}")
print(f"\nNote: High kurtosis indicates fat tails (extreme events)")

In [ ]:
# Visualize Bitcoin prices and returns
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Prices with regime annotations
axes[0].plot(btc['ds'], btc['price'], color=COLORS['blue'], linewidth=1)
axes[0].axvspan(pd.Timestamp('2020-03-01'), pd.Timestamp('2020-04-01'),
                alpha=0.3, color=COLORS['red'], label='COVID-19 Crash')
axes[0].axvspan(pd.Timestamp('2021-11-01'), pd.Timestamp('2022-12-31'),
                alpha=0.2, color=COLORS['orange'], label='Crypto Winter 2022')
axes[0].set_title('Bitcoin Daily Prices (2019-2024) - Real Data', fontweight='bold')
axes[0].set_ylabel('Price (USD)')
axes[0].set_yscale('log')  # Log scale for better visualization
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=2, frameon=False)

# Returns
axes[1].plot(btc['ds'], btc['returns'], color=COLORS['green'], linewidth=0.5)
axes[1].axhline(y=0, color='black', linewidth=0.5, alpha=0.3)
axes[1].set_title('Bitcoin Daily Returns (%)', fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Return (%)')

plt.tight_layout()
plt.subplots_adjust(hspace=0.35)
plt.show()

print("Key Observations:")
print("- COVID-19 crash (March 2020): ~50% drop in weeks")
print("- 2021 bull run to all-time high (~$69K in November 2021)")
print("- 2022 crypto winter: Extended decline to ~$16K")
print("- Clear volatility clustering: Large moves cluster together")

## Step 1: Stationarity Testing

In [ ]:
def test_stationarity(series, name):
    """Run ADF and KPSS tests"""
    print(f"\nStationarity Tests for {name}")
    print("-" * 40)
    
    # ADF Test (H0: unit root exists = non-stationary)
    adf_result = adfuller(series.dropna(), autolag='AIC')
    print(f"ADF Test:")
    print(f"  Statistic: {adf_result[0]:.4f}")
    print(f"  p-value: {adf_result[1]:.4f}")
    print(f"  Conclusion: {'Stationary' if adf_result[1] < 0.05 else 'Non-stationary'}")
    
    # KPSS Test (H0: stationary)
    kpss_result = kpss(series.dropna(), regression='c', nlags='auto')
    print(f"\nKPSS Test:")
    print(f"  Statistic: {kpss_result[0]:.4f}")
    print(f"  p-value: {kpss_result[1]:.4f}")
    print(f"  Conclusion: {'Stationary' if kpss_result[1] > 0.05 else 'Non-stationary'}")
    
    return adf_result[1] < 0.05 and kpss_result[1] > 0.05

# Test prices (expect non-stationary)
prices_stationary = test_stationarity(btc['price'], 'Bitcoin Prices')

# Test returns (expect stationary)
returns_stationary = test_stationarity(btc['returns'].dropna(), 'Bitcoin Returns')

print("\n" + "="*50)
print("CONCLUSION: Use RETURNS for mean modeling (stationary)")
print("But volatility of returns is NOT constant -> GARCH needed!")

## Step 2: ACF/PACF Analysis

In [ ]:
returns = btc['returns'].dropna()

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# ACF/PACF for returns (mean equation)
plot_acf(returns, ax=axes[0, 0], lags=30, alpha=0.05)
axes[0, 0].set_title('ACF: Returns (Mean)', fontweight='bold')

plot_pacf(returns, ax=axes[0, 1], lags=30, alpha=0.05, method='ywm')
axes[0, 1].set_title('PACF: Returns (Mean)', fontweight='bold')

# ACF/PACF for squared returns (volatility)
plot_acf(returns**2, ax=axes[1, 0], lags=30, alpha=0.05)
axes[1, 0].set_title('ACF: Squared Returns (Volatility)', fontweight='bold')

plot_pacf(returns**2, ax=axes[1, 1], lags=30, alpha=0.05, method='ywm')
axes[1, 1].set_title('PACF: Squared Returns (Volatility)', fontweight='bold')

plt.tight_layout()
plt.show()

print("Key Findings:")
print("- Returns: Weak autocorrelation (hard to predict mean)")
print("- Squared returns: STRONG persistence -> GARCH essential!")
print("- This is typical of financial returns: 'weak mean, strong variance'")

## Step 3: ARIMA-GARCH Model

**GARCH(1,1) Model:**
$$\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2$$

Where:
- $\omega$ = constant (long-run variance component)
- $\alpha$ = ARCH effect (reaction to recent shock)
- $\beta$ = GARCH effect (persistence of volatility)
- $\alpha + \beta$ = persistence (how long shocks affect volatility)

In [ ]:
if HAS_ARCH:
    # Fit GARCH(1,1) model with AR(1) mean
    print("Fitting GARCH(1,1) Model for Bitcoin")
    print("="*50)
    
    model = arch_model(returns, vol='Garch', p=1, q=1, mean='AR', lags=1)
    results = model.fit(disp='off')
    
    print(results.summary())
else:
    print("ARCH package not available. Install with: pip install arch")

In [ ]:
if HAS_ARCH:
    # Plot conditional volatility
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # Returns with volatility bands
    cond_vol = results.conditional_volatility
    dates = btc['ds'][1:]
    
    axes[0].plot(dates, returns, color=COLORS['blue'], linewidth=0.5, alpha=0.7, label='Returns')
    axes[0].plot(dates, 2*cond_vol, color=COLORS['red'], linewidth=1, label='+2\u03c3')
    axes[0].plot(dates, -2*cond_vol, color=COLORS['red'], linewidth=1, label='-2\u03c3')
    axes[0].axhline(y=0, color='black', linewidth=0.5, alpha=0.3)
    axes[0].set_title('Bitcoin Returns with GARCH(1,1) Volatility Bands', fontweight='bold')
    axes[0].set_ylabel('Return (%)')
    axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=3, frameon=False)
    
    # Conditional volatility
    axes[1].fill_between(dates, 0, cond_vol, color=COLORS['orange'], alpha=0.7, label='Conditional Volatility')
    axes[1].axvspan(pd.Timestamp('2020-03-01'), pd.Timestamp('2020-04-30'),
                    alpha=0.3, color=COLORS['red'], label='COVID Crash')
    axes[1].set_title('GARCH(1,1) Conditional Volatility', fontweight='bold')
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('Volatility (\u03c3)')
    axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)
    
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.35)
    plt.show()
    
    # Interpret parameters
    alpha = results.params.get('alpha[1]', 0)
    beta = results.params.get('beta[1]', 0)
    
    print("\nGARCH Model Interpretation:")
    print(f"- \u03b1 (ARCH): {alpha:.4f} - Reaction to recent shocks")
    print(f"- \u03b2 (GARCH): {beta:.4f} - Persistence of past volatility")
    print(f"- Persistence (\u03b1+\u03b2): {alpha + beta:.4f}")
    print("\nHigh persistence indicates volatility shocks are long-lasting.")
    if alpha + beta > 0.95:
        print("Warning: Very high persistence - consider IGARCH or long-memory models.")

---
# Case Study 2: Sunspot Cycle Analysis

Sunspots demonstrate:
- **Very long seasonality** (11-year Schwabe cycle)
- **Asymmetric pattern** (rapid rise, slow decline)
- **Variable cycle amplitude**
- Challenge: Standard SARIMA cannot handle such long seasonality

In [ ]:
# Load REAL Sunspot data
sunspots = get_sunspot_data()

print("\nSunspot Data Overview")
print("="*50)
print(f"Period: {sunspots['ds'].min().year} to {sunspots['ds'].max().year}")
print(f"Observations: {len(sunspots)} years")
print(f"\nStatistics:")
print(f"  Mean: {sunspots['y'].mean():.1f}")
print(f"  Std: {sunspots['y'].std():.1f}")
print(f"  Min: {sunspots['y'].min():.0f}")
print(f"  Max: {sunspots['y'].max():.0f}")
print(f"\nKey Feature: ~11-year Schwabe solar cycle")

In [ ]:
# Visualize sunspot data
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Time series
axes[0].plot(sunspots['ds'], sunspots['y'], color=COLORS['blue'], linewidth=1, label='Sunspot Activity')
axes[0].set_title('Yearly Sunspot Numbers (1900-present) - Real Data', fontweight='bold')
axes[0].set_ylabel('Sunspot Count')

# Rolling statistics
window = 11  # 11-year window
rolling_mean = sunspots['y'].rolling(window=window, center=True).mean()
rolling_std = sunspots['y'].rolling(window=window, center=True).std()

axes[1].plot(sunspots['ds'], rolling_mean, color=COLORS['green'], 
             linewidth=2, label=f'{window}-year Rolling Mean')
axes[1].fill_between(sunspots['ds'], rolling_mean - rolling_std, rolling_mean + rolling_std,
                     color=COLORS['green'], alpha=0.2, label='\u00b11 Std Dev')
axes[1].set_title('Rolling Statistics', fontweight='bold')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Sunspot Count')
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)

plt.tight_layout()
plt.subplots_adjust(hspace=0.35)
plt.show()

print("Key Observations:")
print("- Clear ~11-year periodicity (solar cycle)")
print("- Asymmetric shape: rapid rise, gradual decline")
print("- Cycle amplitudes vary significantly")

## ACF Analysis and Spectral Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# ACF showing long cycle
plot_acf(sunspots['y'], ax=axes[0], lags=50, alpha=0.05)
axes[0].axvline(x=11, color=COLORS['red'], linestyle='--', alpha=0.5, label='11-year lag')
axes[0].axvline(x=22, color=COLORS['red'], linestyle='--', alpha=0.5)
axes[0].set_title('ACF: Shows ~11-year Periodicity', fontweight='bold')
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=1, frameon=False)

# FFT to find dominant period
from scipy.fft import fft, fftfreq

y = sunspots['y'].values - sunspots['y'].mean()
n = len(y)
yf = np.abs(fft(y))[:n//2]
xf = fftfreq(n, 1)[:n//2]  # Yearly frequency

# Convert to period in years
period_years = 1 / xf[1:]  # Skip zero frequency
power = yf[1:]

axes[1].plot(period_years[:50], power[:50], color=COLORS['orange'], linewidth=1, label='Power Spectrum')
axes[1].axvline(x=11, color=COLORS['red'], linestyle='--', label='11-year cycle')
axes[1].set_title('Power Spectrum (Periodogram)', fontweight='bold')
axes[1].set_xlabel('Period (years)')
axes[1].set_ylabel('Spectral Power')
axes[1].set_xlim([0, 50])
axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)

plt.tight_layout()
plt.show()

print("Spectral Analysis confirms:")
print("- Dominant period around 11 years")
print("- ACF shows peak correlation at lag 11")

## Modeling with Fourier Terms

For long seasonal periods, we use **Fourier terms** as regressors:

$$y_t = \sum_{k=1}^{K} \left[ a_k \sin\left(\frac{2\pi k t}{m}\right) + b_k \cos\left(\frac{2\pi k t}{m}\right) \right] + \epsilon_t$$

In [ ]:
def add_fourier_terms(df, period, K):
    """Add Fourier terms for seasonality"""
    t = np.arange(len(df))
    for k in range(1, K + 1):
        df[f'sin_{k}'] = np.sin(2 * np.pi * k * t / period)
        df[f'cos_{k}'] = np.cos(2 * np.pi * k * t / period)
    return df

# Prepare data with Fourier terms
sunspot_model = sunspots.copy()
sunspot_model = add_fourier_terms(sunspot_model, period=11, K=3)  # 3 harmonics

# Train/test split
train_size = len(sunspots) - 20  # Hold out last 20 years
train = sunspot_model.iloc[:train_size]
test = sunspot_model.iloc[train_size:]

print(f"Training: {train_size} years")
print(f"Testing: {len(test)} years")
print(f"\nFourier terms added: sin_1, cos_1, sin_2, cos_2, sin_3, cos_3")

In [ ]:
# Fit ARIMA with Fourier regressors
exog_cols = ['sin_1', 'cos_1', 'sin_2', 'cos_2', 'sin_3', 'cos_3']

print("Fitting ARIMA(2,0,1) with Fourier Regressors")
print("="*50)

model = SARIMAX(train['y'], 
                exog=train[exog_cols],
                order=(2, 0, 1),
                enforce_stationarity=False,
                enforce_invertibility=False)
results = model.fit(disp=False)

print(results.summary().tables[1])

In [ ]:
# Forecast
forecast = results.get_forecast(steps=len(test), exog=test[exog_cols])
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

# Metrics
rmse = np.sqrt(mean_squared_error(test['y'], forecast_mean))
mape = np.mean(np.abs((test['y'].values - forecast_mean.values) / (test['y'].values + 1))) * 100

print(f"\nForecast Performance:")
print(f"  RMSE: {rmse:.2f}")
print(f"  MAPE: {mape:.2f}%")

# Plot forecast
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(train['ds'], train['y'], color=COLORS['blue'], linewidth=1, label='Training')
ax.plot(test['ds'], test['y'], color=COLORS['blue'], linewidth=1.5, label='Actual')
ax.plot(test['ds'], forecast_mean, color=COLORS['red'], linewidth=1.5, 
        linestyle='--', label=f'Forecast (RMSE={rmse:.1f})')
ax.fill_between(test['ds'], forecast_ci.iloc[:, 0], forecast_ci.iloc[:, 1],
                color=COLORS['red'], alpha=0.2, label='95% CI')
ax.axvline(x=train['ds'].iloc[-1], color='black', linestyle=':', alpha=0.5)

ax.set_title('Sunspot Forecast: ARIMA with Fourier Terms', fontweight='bold')
ax.set_xlabel('Year')
ax.set_ylabel('Sunspot Count')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=4, frameon=False)

plt.tight_layout()
plt.show()

print("\nNote: Fourier terms capture the 11-year cycle effectively.")

---
# Case Study 3: US Unemployment with Structural Break

The COVID-19 pandemic created an unprecedented structural break:
- Pre-COVID: 3.5% (50-year low)
- Peak: 14.7% (April 2020)
- Recovery: Back to ~3.5% by 2023

This challenges traditional time series models.

In [ ]:
# Load REAL unemployment data
unemp = get_unemployment_data()

print("\nUS Unemployment Data Overview")
print("="*50)
print(f"Period: {unemp['ds'].min().strftime('%Y-%m')} to {unemp['ds'].max().strftime('%Y-%m')}")
print(f"Observations: {len(unemp)} months")
print(f"\nKey Statistics:")
print(f"  Pre-COVID Min: {unemp[unemp['ds'] < '2020-03-01']['y'].min():.1f}%")
print(f"  COVID Peak: {unemp['y'].max():.1f}% (April 2020)")
print(f"  Latest: {unemp['y'].iloc[-1]:.1f}%")
print(f"\nStructural Break: March-April 2020")

In [ ]:
# Visualize unemployment with COVID impact
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Main plot
axes[0].plot(unemp['ds'], unemp['y'], color=COLORS['blue'], linewidth=1.5, label='Unemployment Rate')
axes[0].axvspan(pd.Timestamp('2020-03-01'), pd.Timestamp('2020-05-01'),
                alpha=0.3, color=COLORS['red'], label='COVID-19 Shock')

# Pre-COVID trend
pre_covid = unemp[unemp['ds'] < '2020-03-01']
z1 = np.polyfit(range(len(pre_covid)), pre_covid['y'], 1)
trend_pre = np.polyval(z1, range(len(pre_covid)))
axes[0].plot(pre_covid['ds'], trend_pre, color=COLORS['gray'], linewidth=2, 
             linestyle='--', label='Pre-COVID Trend')

axes[0].set_title('US Unemployment Rate (2015-2023): COVID-19 Structural Break - Real Data', fontweight='bold')
axes[0].set_ylabel('Unemployment Rate (%)')
axes[0].set_ylim([0, 16])
axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=3, frameon=False)

# Month-over-month change
unemp_change = unemp['y'].diff()
colors_bar = [COLORS['green'] if x < 0 else COLORS['red'] for x in unemp_change]
axes[1].bar(unemp['ds'], unemp_change, color=colors_bar, width=20, alpha=0.7)
axes[1].axhline(y=0, color='black', linewidth=0.5)
axes[1].set_title('Month-over-Month Change', fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Change (pp)')

plt.tight_layout()
plt.subplots_adjust(hspace=0.35)
plt.show()

print("Key Observations:")
print("- April 2020: Largest single-month increase in history")
print("- Recovery was rapid but took ~3 years to return to pre-COVID levels")
print("- Traditional ARIMA would struggle with such an extreme outlier")

## Prophet Model for Structural Breaks

Prophet handles structural breaks through **changepoint detection**:
- Automatically identifies where the trend changes
- Adjustable `changepoint_prior_scale` controls flexibility
- Can add known changepoints (like COVID)

In [ ]:
if HAS_PROPHET:
    # Prepare data for Prophet
    unemp_prophet = unemp.copy()
    
    # Train/test split (hold out last 12 months)
    train_unemp = unemp_prophet.iloc[:-12]
    test_unemp = unemp_prophet.iloc[-12:]
    
    print("Fitting Prophet Model with Changepoint Detection")
    print("="*50)
    
    # Model with flexible changepoints
    prophet_model = Prophet(
        changepoint_prior_scale=0.5,  # More flexible for extreme changes
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        changepoints=['2020-03-01', '2020-04-01', '2020-06-01']  # COVID events
    )
    prophet_model.fit(train_unemp)
    
    # Forecast
    future = prophet_model.make_future_dataframe(periods=12, freq='MS')
    forecast = prophet_model.predict(future)
    
    # Extract test predictions
    pred_test = forecast['yhat'].iloc[-12:].values
    
    # Metrics
    rmse = np.sqrt(mean_squared_error(test_unemp['y'], pred_test))
    mape = np.mean(np.abs((test_unemp['y'].values - pred_test) / test_unemp['y'].values)) * 100
    
    print(f"\nProphet Forecast Performance:")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAPE: {mape:.2f}%")
else:
    print("Prophet not available. Install with: pip install prophet")

In [ ]:
if HAS_PROPHET:
    # Plot Prophet results
    fig, axes = plt.subplots(2, 1, figsize=(12, 8))
    
    # Forecast plot
    axes[0].plot(unemp['ds'], unemp['y'], color=COLORS['blue'], linewidth=1.5, label='Actual')
    axes[0].plot(forecast['ds'], forecast['yhat'], color=COLORS['orange'], 
                 linewidth=1.5, linestyle='--', label='Prophet Forecast')
    axes[0].fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'],
                        color=COLORS['orange'], alpha=0.2, label='95% CI')
    axes[0].axvline(x=train_unemp['ds'].iloc[-1], color='black', linestyle=':', alpha=0.5)
    
    # Mark changepoints
    for i, cp in enumerate(prophet_model.changepoints):
        if i == 0:
            axes[0].axvline(x=cp, color=COLORS['red'], linestyle='--', alpha=0.5, linewidth=1, label='Changepoints')
        else:
            axes[0].axvline(x=cp, color=COLORS['red'], linestyle='--', alpha=0.5, linewidth=1)
    
    axes[0].set_title('Prophet Model with Changepoint Detection', fontweight='bold')
    axes[0].set_ylabel('Unemployment Rate (%)')
    axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=4, frameon=False)
    
    # Trend component
    axes[1].plot(forecast['ds'], forecast['trend'], color=COLORS['green'], linewidth=2, label='Trend')
    for i, cp in enumerate(prophet_model.changepoints):
        if i == 0:
            axes[1].axvline(x=cp, color=COLORS['red'], linestyle='--', alpha=0.5, linewidth=1, label='Changepoints')
        else:
            axes[1].axvline(x=cp, color=COLORS['red'], linestyle='--', alpha=0.5, linewidth=1)
    axes[1].set_title('Prophet Trend Component (with Changepoints)', fontweight='bold')
    axes[1].set_xlabel('Date')
    axes[1].set_ylabel('Trend')
    axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)
    
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.35)
    plt.show()
    
    print("\nProphet Changepoints detected:")
    for i, cp in enumerate(prophet_model.changepoints):
        print(f"  {i+1}. {cp.strftime('%Y-%m-%d')}")

---
# Summary: Model Selection Guide

## Decision Flowchart

```
Time Series Data
    |
    v
[Is data stationary?] --No--> Apply differencing/transformations
    |
    Yes
    v
[Financial returns?] --Yes--> ARIMA-GARCH
    |
    No
    v
[Seasonality present?]
    |         |
   Yes        No
    |         |
    v         v
[Long season?]   ARIMA
(>12 periods)
    |    |
   Yes   No
    |    |
    v    v
Fourier  SARIMA
Terms
    |
    v
[Structural breaks?] --Yes--> Prophet/Piecewise models
```

In [ ]:
print("""
\u2554\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2557
\u2551                    TIME SERIES MODEL SELECTION GUIDE                  \u2551
\u2560\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2563
\u2551                                                                      \u2551
\u2551  DATA TYPE              RECOMMENDED MODEL         SPECIAL CASE       \u2551
\u2551  \u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500  \u2551
\u2551                                                                      \u2551
\u2551  Crypto/Financial       ARIMA-GARCH               Extreme volatility \u2551
\u2551  (Bitcoin, stocks)      Focus on volatility       clustering         \u2551
\u2551                                                                      \u2551
\u2551  Long seasonality       ARIMA + Fourier           Period > 12        \u2551
\u2551  (Sunspots, climate)    terms as regressors       years              \u2551
\u2551                                                                      \u2551
\u2551  Structural breaks      Prophet with              Changepoint        \u2551
\u2551  (COVID, crises)        changepoints              detection          \u2551
\u2551                                                                      \u2551
\u2551  Standard seasonality   SARIMA                    Period \u2264 12        \u2551
\u2551  (monthly, quarterly)                                                \u2551
\u2551                                                                      \u2551
\u2551  Multiple seasonality   Prophet, TBATS            Daily + weekly +   \u2551
\u2551  (hourly data)                                    annual             \u2551
\u2551                                                                      \u2551
\u2560\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2563
\u2551                         WORKFLOW STEPS                               \u2551
\u2551  1. Visualize and explore the data                                   \u2551
\u2551  2. Test for stationarity (ADF, KPSS)                                \u2551
\u2551  3. Apply transformations if needed (log, diff)                      \u2551
\u2551  4. Identify patterns (ACF/PACF, decomposition)                      \u2551
\u2551  5. Fit candidate models                                             \u2551
\u2551  6. Check diagnostics (residuals, Ljung-Box)                         \u2551
\u2551  7. Compare with out-of-sample forecast                              \u2551
\u2551  8. Select best model for the task                                   \u2551
\u255a\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u2550\u255d
""")

In [ ]:
# Final summary table
print("\nChapter 10 Case Studies Summary")
print("="*75)
print(f"{'Dataset':<20} {'Challenge':<25} {'Best Model':<20} {'Key Insight'}")
print("-"*75)
print(f"{'Bitcoin':<20} {'Extreme volatility':<25} {'ARIMA-GARCH(1,1)':<20} {'Focus on volatility, not mean'}")
print(f"{'Sunspots':<20} {'11-year cycle':<25} {'ARIMA + Fourier':<20} {'Long seasonality handling'}")
print(f"{'US Unemployment':<20} {'COVID structural break':<25} {'Prophet':<20} {'Changepoint detection'}")
print("-"*75)

print("\n" + "="*75)
print("KEY TAKEAWAYS")
print("="*75)
print("""
1. VOLATILITY MODELING (Bitcoin)
   - Financial returns show weak mean predictability but strong volatility patterns
   - GARCH captures volatility clustering: large moves followed by large moves
   - Always examine squared returns for ARCH effects

2. LONG SEASONALITY (Sunspots)
   - When seasonal period > 12, traditional SARIMA is computationally challenging
   - Fourier terms as regressors provide an elegant solution
   - Use spectral analysis to identify the true period

3. STRUCTURAL BREAKS (Unemployment)
   - Extreme events (COVID, financial crises) require flexible models
   - Prophet's changepoint detection handles regime changes
   - Consider using post-break data only for ARIMA

4. GENERAL PRINCIPLES
   - Always start with visualization and stationarity testing
   - Match the model to the data characteristics
   - Validate with out-of-sample forecasting
   - Simple models often work better than complex ones
""")

---

## Data Sources

- **Bitcoin**: Yahoo Finance (BTC-USD)
- **Sunspots**: Classic Wolfer sunspot dataset from statsmodels
- **US Unemployment**: Bureau of Labor Statistics via FRED (series UNRATE)

---

**End of Chapter 10: Comprehensive Review**

*Course: Time Series Analysis and Forecasting*  
*Bucharest University of Economic Studies*